<a href="https://colab.research.google.com/github/Gh5al/CDMO/blob/main/MIP/MIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install highspy
!apt-get install coinor-cbc
!apt-get install -y -qq glpk-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 35 not upgraded.
Need to get 2,908 kB of archives.
After this operation, 8,310 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcoinutils3v5 amd64 2.11.4+repack1-2 [465 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libosi1v5 amd64 0.108.6+repack1-2 [275 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libclp1 amd64 1.17.5+repack1-1 [937 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcgl1 amd64 0.60.3+repack1-3 [420 kB]
Get:5 ht

3d decision variable + MTZ constraint to prevent subtours<br>
3 solvers are used:
- GLPK
- CBC
- HiGHS

In [2]:
from pyomo.environ import *
from pyomo.contrib.appsi.solvers import Cbc, Highs
import time

In [32]:
filename = "./inst13.dat"
dist = []
with open(filename,'r') as f:
  m = int(f.readline().strip())
  n = int(f.readline().strip())
  capacity = [int(s) for s in f.readline().strip().split()]
  size = [int(s) for s in f.readline().strip().split()]
  for i in range(n+1):
    dist.append([int(s) for s in f.readline().strip().split()])

print(m,n)
print(capacity)
print(size)
print(dist)

3 47
[300, 200, 200]
[12, 8, 16, 5, 12, 5, 13, 20, 13, 18, 7, 6, 9, 9, 4, 25, 5, 17, 3, 16, 25, 21, 14, 19, 14, 6, 16, 9, 20, 13, 10, 16, 19, 22, 14, 10, 11, 15, 13, 15, 8, 22, 24, 3, 25, 19, 21]
[[0, 60, 141, 22, 41, 137, 77, 48, 92, 105, 113, 103, 82, 15, 79, 24, 98, 69, 82, 30, 105, 89, 57, 94, 75, 50, 127, 16, 36, 77, 57, 70, 51, 101, 88, 38, 83, 108, 81, 124, 54, 131, 99, 70, 112, 162, 94, 64], [60, 0, 83, 82, 99, 81, 53, 106, 34, 61, 57, 43, 140, 63, 23, 56, 84, 59, 142, 88, 45, 79, 27, 34, 33, 46, 107, 58, 94, 17, 29, 26, 109, 51, 34, 30, 45, 48, 23, 64, 50, 73, 41, 26, 68, 104, 34, 36], [141, 83, 0, 129, 182, 4, 64, 189, 63, 36, 28, 126, 223, 126, 106, 139, 167, 142, 125, 171, 128, 52, 110, 117, 116, 129, 24, 125, 177, 100, 84, 71, 192, 40, 53, 103, 58, 131, 90, 147, 87, 70, 100, 71, 29, 101, 117, 77], [22, 82, 129, 0, 55, 125, 65, 60, 80, 93, 101, 125, 94, 19, 101, 46, 120, 91, 60, 42, 127, 77, 79, 116, 97, 72, 115, 24, 48, 99, 53, 58, 63, 89, 76, 52, 71, 130, 89, 146, 42, 119

In [33]:
#use the xkij and a ordering constraint
model = ConcreteModel()
model.K = Set(initialize=range(m)) #couriers
model.I = Set(initialize=range(n)) #items
model.loc=Set(initialize=range(n+1)) #locations

#variables
model.x = Var(model.K, model.loc,model.loc,domain=Binary) #successor variable
model.o = Var(model.K, model.I,domain=PositiveIntegers)  # order variable to prevent subtour
model.dist = Var(model.K, domain=PositiveIntegers) #memorize the distance for each courier
model.obj = Var(domain=PositiveIntegers)  # max distance to minimize

#constraints
#each item should be delivered by only one courier
def one_item_delivery_rule(model,j):
    return sum(model.x[k,i,j] for k in model.K for i in model.loc if i!=j) == 1
model.one_item_delivery = Constraint(model.I, rule=one_item_delivery_rule)

#enter and exit from node, number of times exit a node i = number of times enter a node i
def enter_exit_rule(model,k,i):
    return sum(model.x[k,i,j] for j in model.loc) == sum(model.x[k,j,i] for j in model.loc)
model.enter_exit = Constraint(model.K, model.loc, rule=enter_exit_rule)

#each courier start from depot exactly once
def leave_depot_rule(model,k):
    return sum(model.x[k,n,j] for j in model.I) == 1
model.depot_leave = Constraint(model.K, rule=leave_depot_rule)

#each courier return to depot exactly once
def return_depot_rule(model,k):
    return sum(model.x[k,i,n] for i in model.I) == 1
model.depot_return = Constraint(model.K, rule=return_depot_rule)

#avoid self-loop
def self_loop_rule(model,k,i):
    return model.x[k,i,i] == 0
model.self_loop = Constraint(model.K, model.loc, rule=self_loop_rule)


#capacity constraint
def capacity_rule(model,k):
    return sum(size[i]*(model.x[k,i,j]) for i in model.I for j in model.loc if i!=j) <= capacity[k]
model.capacity = Constraint(model.K, rule=capacity_rule)

#prevent subtours with MTZ constraint, used in TSP problem
#https://phabe.ch/2021/09/19/tsp-subtour-elimination-by-miller-tucker-zemlin-constraint/
model.order = ConstraintList()
for k in model.K:
    for i in model.I:
      for j in model.I:
        if i != j:
      #use Big M notation(M=2*n)
          model.order.add(model.o[k, j] - model.o[k,i] >= 1 -(1-model.x[k,i,j])*2*n)

#compute distance
def total_distance_rule(model,k):
    return model.dist[k] == sum(dist[i][j] * model.x[k, i, j] for i in model.loc for j in model.loc if i != j)
model.total_distance_constraint = Constraint(model.K, rule=total_distance_rule)

#take the max distance travelled by any courier as obj
def max_distance_constraint_rule(model, k):
    return model.obj >= model.dist[k]
model.max_distance_constraint = Constraint(model.K, rule=max_distance_constraint_rule)

#lowerbound constraint
def lower_bound_rule(model,i):
    return model.obj >= (dist[n][i] + dist[i][n])
model.lower_bound = Constraint(model.I, rule=lower_bound_rule)
#objective
model.objective = Objective(expr = model.obj, sense=minimize)



In [34]:
#GLPK SOLVER
solver = SolverFactory('glpk')
solver.options['tmlim'] = 300
results = solver.solve(model,tee=False)
print(results)
print(model.obj.value)

KeyboardInterrupt: 

In [24]:
#CBC SOLVER
start_time = time.time()
solver = Cbc()
solver.config.time_limit = 300
results = solver.solve(model)
final_time = time.time() - start_time
print(results)
print(final_time)
print(model.obj.value)

termination_condition: TerminationCondition.optimal
best_feasible_objective: 16.0
best_objective_bound: -inf
0.34814929962158203
16.0


In [35]:
#HIGHS SOLVER
start_time = time.time()
solver = Highs()
solver.config.time_limit = 300
results = solver.solve(model)
final_time = time.time() - start_time
print(results)
print(final_time)
print(model.obj.value)

termination_condition: TerminationCondition.maxTimeLimit
best_feasible_objective: 592.0
best_objective_bound: 303.0
301.0201504230499
592.0


In [39]:
print(model.I.data())
sol = []
for k in model.K:
    sub_sol = []
    print(f"Courier {k}:")
    for j in model.loc:
      if value(model.x[k,n,j]) > 0.5:
        first = j
        sub_sol.append(first+1)
    for i in model.loc:
        for j in model.loc:
            if value(model.x[k, i, j]) > 0.5:
                print(f"travels from {i} to {j}")
    succ=0
    prec=first
    while(True):
      for i in model.loc:
        if value(model.x[k,prec,i]>0.5):
          #print(f" from {prec} to {i}")
          if i == n:
            succ=n
          prec=i
          break
      if succ == n:
        break
      sub_sol.append(prec+1)
    print(sub_sol)
    sol.append(sub_sol)
print(sol)


(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46)
Courier 0:
travels from 1 to 39
travels from 4 to 47
travels from 7 to 4
travels from 11 to 37
travels from 12 to 7
travels from 13 to 27
travels from 15 to 28
travels from 19 to 32
travels from 20 to 35
travels from 23 to 46
travels from 27 to 15
travels from 28 to 19
travels from 32 to 12
travels from 35 to 13
travels from 37 to 20
travels from 39 to 23
travels from 46 to 11
travels from 47 to 1
[2, 40, 24, 47, 12, 38, 21, 36, 14, 28, 16, 29, 20, 33, 13, 8, 5]
Courier 1:
travels from 8 to 38
travels from 14 to 22
travels from 16 to 17
travels from 17 to 14
travels from 22 to 30
travels from 24 to 47
travels from 25 to 16
travels from 29 to 24
travels from 30 to 8
travels from 33 to 29
travels from 38 to 42
travels from 41 to 45
travels from 42 to 41
travels from 45 to 33
travels from 47 to 25
[26, 17, 18, 15

In [38]:
import json
import os
import math

# Example values
instance_id = 3
approach = "MIP"
approach_name = "highs"

if final_time<300:
  optimal = "true"
  time = math.floor(final_time)
else:
  optimal = "false"
  time = 300

objective_value = model.obj.value
solution = sol.copy()

# Create JSON structure
res = {
    approach_name: {
        "time": time,
        "optimal": optimal,
        "obj": int(objective_value),
        "sol": sol
    }
}
# Make sure directory exists
out_dir = f"res/{approach}"
os.makedirs(out_dir,exist_ok=True)

# Save to file
with open(f"{out_dir}/{instance_id}.json", "w") as f:
    json.dump(res, f)
